In [108]:
#Ejercicio 4
import simpy
import random

#Unidad de tiempo: milisegundo
#0.7s son  700ms
#0.8s son  800ms
#1.0s son 1000ms
#4.0s son 4000ms

#Configuracion
#Cantidad de solicitudes a procesar
cantidad_solicitudes = 100000
#Cantidad de bases de datos a utilizar 1 o 2
cantidad_bases = 2

#Modo DEBUG, imprime las listas de tiempos
modo_debug = False

In [109]:
def atenderSolicitud(env, baseDatos, mu, id_solicitud): 
    
    t_atencion = random.expovariate(mu)

    tiempoInicio = env.now
    db = baseDatos.request()
    yield db
    tiempoEspera = env.now - tiempoInicio
    tiemposEsperas.append(tuple((id_solicitud, tiempoEspera)))
    yield env.timeout(t_atencion)
    baseDatos.release(db)
    tiempoTotal = env.now - tiempoInicio
    tiemposTotales.append(tuple((id_solicitud, tiempoTotal)))

In [110]:
class generadorSolicitudes:
    
    def __init__(self, env, cantidad, bases):
        
        self.env = env
        self.action = env.process(self.generarSolicitudes(env, cantidad, bases))

    def generarSolicitudes(self, env, cantidad, bases):

        baseDatos1 = simpy.Resource(env, capacity=1)
        baseDatos2 = simpy.Resource(env, capacity=1)
        
        id_solicitud = 0
        for i in range(cantidad):
            
            if (bases == 1):

                mu1 = 800
                solicitud = env.process(atenderSolicitud(env, baseDatos1, mu1, id_solicitud))

            else:
                
                mu1 =  700
                mu2 = 1000

                if (random.uniform(0, 1) > 0.6):
                    
                    solicitud = env.process(atenderSolicitud(env, baseDatos2, mu2, id_solicitud))
            
                else:
                    
                    solicitud = env.process(atenderSolicitud(env, baseDatos1, mu1, id_solicitud))

            tiempoEntreSolicitudes = random.expovariate(4000)
            yield env.timeout(tiempoEntreSolicitudes)
            
            id_solicitud = id_solicitud + 1

In [111]:
#Listas para guardar los tiempos
tiemposEsperas = []
tiemposTotales = []

#Inicializacion del entorno
env = simpy.Environment()

#Generacion de solicitudes y ejecucion de la simulacion
solicitudes = generadorSolicitudes(env, cantidad_solicitudes, cantidad_bases)
env.run()

#Calculo de elementos con tiempo de espera 0
cantSinEspera = 0
for t in tiemposEsperas:
    if t[1] == 0:
        cantSinEspera = cantSinEspera + 1
        
#Impresion de resultados
print("Cantidad de solicitudes procesadas: %d" % len(tiemposEsperas))
print("Cantidad de solicitudes sin espera: %d" % cantSinEspera)
print("Fracción de solicitudes sin espera: %f" %(cantSinEspera/len(tiemposEsperas)))
print("Media de espera (ms): %f" %(sum([t[1] for t in tiemposEsperas])/len(tiemposEsperas)))
print("Media de espera y procesamiento (ms): %f" %(sum([t[1] for t in tiemposTotales])/len(tiemposTotales)))

#Impresion de listas de tiempos
if (modo_debug == True):
    
    print(tiemposEsperas)
    print(tiemposTotales)

Cantidad de solicitudes procesadas: 100000
Cantidad de solicitudes sin espera: 5
Fracción de solicitudes sin espera: 0.000050
Media de espera (ms): 21.478499
Media de espera y procesamiento (ms): 21.479761


In [ ]:
#Resultados alternativa 1 base
#Cantidad de solicitudes procesadas: 100000
#Cantidad de solicitudes sin espera: 1
#Fracción de solicitudes sin espera: 0.000010
#Media de espera (ms): 49.842979
#Media de espera y procesamiento (ms): 49.844222

#Resultados alternativa 2 bases
#Cantidad de solicitudes procesadas: 100000
#Cantidad de solicitudes sin espera: 5
#Fracción de solicitudes sin espera: 0.000050
#Media de espera (ms): 21.478499
#Media de espera y procesamiento (ms): 21.479761
